In [ ]:
# required modules 
import random 
import json 
import pickle 
import numpy as np 
import nltk 
from keras.models import load_model 
from nltk.stem import WordNetLemmatizer 
from datetime import datetime
now = datetime.now()

In [ ]:
lemmatizer = WordNetLemmatizer() 
  
# loading the files we made previously 
intents = json.loads(open("intents/intents.json").read()) 
words = pickle.load(open('words.pkl', 'rb'))
#print(words)
classes = pickle.load(open('classes.pkl', 'rb')) 
model = load_model('chatbotmodel.h5') 

This cell finds the output of any arithematic equation  

In [ ]:
import ast

def calculate_expression(expression):
    try:
        # Parse the expression
        parsed_expr = ast.parse(expression, mode='eval')
        # Evaluate the parsed expression
        result = eval(compile(parsed_expr, '<string>', 'eval'))
        return result
    except Exception as e:
        return f"Error: {e}"

This cell recognizes the keywords related to real time or calculation and return the output

In [ ]:
def genral_question(intent, s):
    #print(intent)
    if(intent == 'date'):
        return datetime.today()
    
    elif(intent == 'time'):
        return now.strftime("%H:%M:%S")
    
    elif(intent == 'weekday'):
        return now.weekday()
    
    elif(intent == 'calculation'):
        for i in range(len(s)):
            if(s[i] == '-' or s[i].isnumeric()):
                return calculate_expression(s[i:])
    else:
        return " "
    

This cell coverts the message to the updated message which does not contains any dulicates and not having the grammar involved

In [ ]:
def clean_up_sentences(sentence): 
    sentence_words = nltk.word_tokenize(sentence) 
    sentence_words = [lemmatizer.lemmatize(word)  
                      for word in sentence_words] 
    return sentence_words 

This cell finds the words present in the message and finds the words which are present in the intents and mark it as 1

In [ ]:
def bagw(sentence): 
    sentence = sentence.lower()
    # separate out words from the input sentence 
    sentence_words = clean_up_sentences(sentence) 
    bag = [0]*len(words) 
    for w in sentence_words: 
        for i, word in enumerate(words): 
  
            # check whether the word 
            # is present in the input as well 
            if word == w: 
  
                # as the list of words 
                # created earlier. 
                bag[i] = 1
  
    # return a numpy array 
    return np.array(bag) 

This cell predicts the class for the message given by the user 

In [ ]:
def predict_class(sentence): 
    bow = bagw(sentence) 
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res)  
               if r > ERROR_THRESHOLD] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_list = [] 
    for r in results: 
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])}) 
    
    return return_list 

This cell return the responce to the user after finding the best suitable answer from the model

In [ ]:
def get_response(intents_list, intents_json): 
    tag = intents_list[0]['intent'] 
    list_of_intents = intents_json['intents'] 
    result = "I can't undersatnd!" 
    for i in list_of_intents: 
        if i['tag'] == tag: 
            result = random.choice(i['responses']) 
            break
    return result 

print("Chatbot is up!") 

In [ ]:
while True: 
    message = input("") 
    if(message == 'exit'):
        break
    ints = predict_class(message) 
    res = get_response(ints, intents) 
    res += str(genral_question(ints[0]['intent'], message))
    print(res)